In [1]:
import tempfile
import pandas as pd

from gtfparse import read_gtf
from Bio import SeqIO
from datapaths import *
from tqdm.notebook import tqdm

pd.set_option('display.max_rows', 5)
pd.set_option('display.max_columns', None)

In [2]:
gtf = read_gtf(F_GENCODE)
gtf

/home/fpavlov/miniconda3/envs/mirna/lib/python3.11/site-packages/gtfparse/read_gtf.py:82: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  chunk_iterator = pd.read_csv(
/home/fpavlov/miniconda3/envs/mirna/lib/python3.11/site-packages/gtfparse/read_gtf.py:82: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  chunk_iterator = pd.read_csv(
INFO:root:Extracted GTF attributes: ['gene_id', 'gene_type', 'gene_name', 'level', 'mgi_id', 'havana_gene', 'transcript_id', 'transcript_type', 'transcript_name', 'transcript_support_level', 'tag', 'havana_transcript', 'exon_number', 'exon_id', 'protein_id', 'ccdsid', 'ont']


,seqname,source,feature,start,end,score,strand,frame,gene_id,gene_type,gene_name,level,mgi_id,havana_gene,transcript_id,transcript_type,transcript_name,transcript_support_level,tag,havana_transcript,exon_number,exon_id,protein_id,ccdsid,ont
0,chr1,HAVANA,gene,3073253,3074322,NaN,+,0,ENSMUSG00000102693.1,TEC,4933401J01Rik,2,MGI:1918292,OTTMUSG00000049935.1,,,,,,,,,,,
1,chr1,HAVANA,transcript,3073253,3074322,NaN,+,0,ENSMUSG00000102693.1,TEC,4933401J01Rik,2,MGI:1918292,OTTMUSG00000049935.1,ENSMUST00000193812.1,TEC,4933401J01Rik-201,NA,basic,OTTMUST00000127109.1,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1302161,chrM,ENSEMBL,transcript,15356,15422,NaN,-,0,ENSMUSG00000064372.1,Mt_tRNA,mt-Tp,3,MGI:102478,,ENSMUST00000082423.1,Mt_tRNA,mt-Tp-201,NA,basic,,,,,,
1302162,chrM,ENSEMBL,exon,15356,15422,NaN,-,0,ENSMUSG00000064372.1,Mt_tRNA,mt-Tp,3,MGI:102478,,ENSMUST00000082423.1,Mt_tRNA,mt-Tp-201,NA,basic,,1,ENSMUSE00000521550.1,,,


In [3]:
print(gtf['gene_id'].nunique())
print(gtf[gtf['gene_type']=='protein_coding']['gene_id'].nunique())
print(gtf['transcript_id'].nunique())
print(gtf[gtf['gene_type']=='protein_coding']['transcript_id'].nunique())

55401
21859
81541
45557


In [4]:
flipon_to_data = pd.read_table(F_FLIPON_TO_DATA)
flipon_to_data


,Flipon,Coordinates,Strand,Gene Feature,Gene Name,Gene Strand,Gene Type,Gene ID,Transcript ID,c. M miR (+),c. M miR (-),c. MJ miR (+),c. MJ miR (-),c. J miR (+),c. J miR (-),all M miR (+),all M miR (-),all MJ miR (+),all MJ miR (-),all J miR (+),all J miR (-),cCRE (+-200bp),LINE/LTR (+-200bp),Feature Group
0,g4,chr1:3014794-3014871,-,Distal Intergenic,4933401J01Rik,+,TEC,ENSMUSG00000102693.1,ENSMUST00000193812.1,miR-328 (1),NaN,miR-486 (1),NaN,NaN,NaN,"miR-6994 (1), miR-129b (1)","miR-92a-2 (1), miR-698/miR-7078 (1), miR-3572 ...",miR-486a/miR-486b (1),"miR-3076 (1), miR-6418 (1), miR-6987 (1)",NaN,miR-7649 (1),NaN,"LINE@L1Md_F2(-), LINE@L1VL1(+)",LINE/LTR
1,g4,chr1:3099888-3099963,+,Promoter (2-3kb),Gm26206,+,snRNA,ENSMUSG00000064842.1,ENSMUST00000082908.1,NaN,miR-328 (1),NaN,NaN,NaN,NaN,"miR-365-1/miR-365-2 (1), miR-698/miR-7078 (1),...","miR-221 (1), miR-129b (1)","miR-6418 (1), miR-7067 (1)",NaN,NaN,NaN,NaN,"LINE@L1Md_F2(+), LTR@MTC(+)",LINE/LTR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77706,z-dna,chrY:4203100-4203146,.,Distal Intergenic,Gm28191,+,unprocessed_pseudogene,ENSMUSG00000100300.1,ENSMUST00000189112.1,NaN,NaN,miR-493 (1),NaN,NaN,NaN,"miR-466d/miR-466i (4), miR-466l (5)",NaN,miR-493 (1),NaN,NaN,NaN,NaN,LINE@L1_Mur1(-),LINE/LTR
77707,z-dna,chrY:4207093-4207113,.,Distal Intergenic,Gm28191,+,unprocessed_pseudogene,ENSMUSG00000100300.1,ENSMUST00000189112.1,NaN,NaN,NaN,NaN,NaN,NaN,"miR-466a/miR-466e/miR-466p (1), miR-466d/miR-4...",NaN,NaN,NaN,NaN,NaN,NaN,"LINE@L1_Mus3(+), LINE@L1Md_A(+), LTR@ORR1B1(+)",LINE/LTR


In [5]:
from itertools import combinations

mir_groups = ['M', 'MJ', 'J']
mir_group_combinations = list()
for r in range(1,4):
    cur_combinations = combinations(mir_groups, r=r)
    for groups in cur_combinations:
        temp = []
        for group in groups:
            temp += [f'c. {group} miR (+)', f'c. {group} miR (-)']
        mir_group_combinations.append(('+'.join(groups),temp))
mir_group_combinations

[('M', ['c. M miR (+)', 'c. M miR (-)']),
 ('MJ', ['c. MJ miR (+)', 'c. MJ miR (-)']),
 ('J', ['c. J miR (+)', 'c. J miR (-)']),
 ('M+MJ', ['c. M miR (+)', 'c. M miR (-)', 'c. MJ miR (+)', 'c. MJ miR (-)']),
 ('M+J', ['c. M miR (+)', 'c. M miR (-)', 'c. J miR (+)', 'c. J miR (-)']),
 ('MJ+J', ['c. MJ miR (+)', 'c. MJ miR (-)', 'c. J miR (+)', 'c. J miR (-)']),
 ('M+MJ+J',
  ['c. M miR (+)',
   'c. M miR (-)',
   'c. MJ miR (+)',
   'c. MJ miR (-)',
   'c. J miR (+)',
   'c. J miR (-)'])]

In [6]:
result_df = pd.DataFrame()

data = flipon_to_data.copy()
data_group = data['Flipon'].value_counts().sort_index()
data_all = data.shape[0]
result_df = pd.concat([result_df, pd.Series([*data_group.values, data_all], index=[*data_group.index, 'Any flipon'], name='# of flipons in genome')], axis=1)

data = flipon_to_data[flipon_to_data['Gene Feature']=="Promoter (<=1kb)"]
data_group = data['Flipon'].value_counts().sort_index()
data_all = data.shape[0]
result_df = pd.concat([result_df, pd.Series([*data_group.values, data_all], index=[*data_group.index, 'Any flipon'], name='# of flipons in PP')], axis=1)

data = flipon_to_data[(flipon_to_data['Gene Feature']=="Promoter (<=1kb)") & (flipon_to_data['Gene Type']=='protein_coding')]
data_group = data['Flipon'].value_counts().sort_index()
data_all = data.shape[0]
result_df = pd.concat([result_df, pd.Series([*data_group.values, data_all], index=[*data_group.index, 'Any flipon'], name='# of flipons in PP (coding)')], axis=1)

result_df_2 = pd.DataFrame()
for groups, mir_group_combination in mir_group_combinations:
    data = flipon_to_data[(flipon_to_data['Gene Feature']=="Promoter (<=1kb)") & (flipon_to_data[mir_group_combination].notna().sum(axis=1)>0)]
    data_group = data['Flipon'].value_counts().sort_index()
    data_all = data.shape[0]
    result_df_2 = pd.concat([result_df_2, pd.Series([*data_group.values, data_all], index=[*data_group.index, 'Any flipon'], name=groups)], axis=1)
result_df_2.columns = [['# flipons bound by c. miR in PP'] * result_df_2.shape[1], result_df_2.columns]
result_df_2

result_df_3 = pd.DataFrame()
for groups, mir_group_combination in mir_group_combinations:
    data = flipon_to_data[(flipon_to_data['Gene Feature']=="Promoter (<=1kb)") & (flipon_to_data['Gene Type']=='protein_coding') & (flipon_to_data[mir_group_combination].notna().sum(axis=1)>0)]
    data_group = data['Flipon'].value_counts().sort_index()
    data_all = data.shape[0]
    result_df_3 = pd.concat([result_df_3, pd.Series([*data_group.values, data_all], index=[*data_group.index, 'Any flipon'], name=groups)], axis=1)
result_df_3.columns = [['# flipons bound by c. miR in PP (coding)'] * result_df_3.shape[1], result_df_3.columns]
result_df_3

display(result_df, result_df_2, result_df_3)

,# of flipons in genome,# of flipons in PP,# of flipons in PP (coding)
g4,20253,6565,5521
h-dna,17100,1767,1156
sidd,15296,860,394
z-dna,25059,9365,7799
Any flipon,77708,18557,14870


# flipons bound by c. miR in PP                                    
                                         M    MJ    J  M+MJ   M+J  MJ+J M+MJ+J
g4                                    2674  1949  339  3758  2877  2181   3908
h-dna                                  269   400   85   586   346   464    643
sidd                                   790   214   49   805   795   253    810
z-dna                                 1468  2311  497  3367  1893  2680   3687
Any flipon                            5201  4874  970  8516  5911  5578   9048

# flipons bound by c. miR in PP (coding)                         \
                                                  M    MJ    J  M+MJ   M+J   
g4                                             2284  1645  281  3198  2449   
h-dna                                           188   273   61   408   243   
sidd                                            353    98   24   362   357   
z-dna                                          1170  1906  418  2756  1528   
Any flipon                                     3995  3922  784  6724  4577   

                         
            MJ+J M+MJ+J  
g4          1830   3316  
h-dna        319    448  
sidd         117    366  
z-dna       2218   3029  
Any flipon  4484   7159

In [8]:
result_df = pd.DataFrame()

data = flipon_to_data.copy()
data_group = data.groupby('Flipon')['Gene ID'].nunique().sort_index()
data_all = data['Gene ID'].nunique()
result_df = pd.concat([result_df, pd.Series([*data_group.values, data_all], index=[*data_group.index, 'Any flipon'], name='# of genes with ≥1 flipon')], axis=1)

data = flipon_to_data[(flipon_to_data['Gene Type']=='protein_coding')]
data_group = data.groupby('Flipon')['Gene ID'].nunique().sort_index()
data_all = data['Gene ID'].nunique()
result_df = pd.concat([result_df, pd.Series([*data_group.values, data_all], index=[*data_group.index, 'Any flipon'], name='# of genes with ≥1 flipon (coding)')], axis=1)

result_df_2 = pd.DataFrame()
for groups, mir_group_combination in mir_group_combinations:
    data = flipon_to_data[(flipon_to_data[mir_group_combination].notna().sum(axis=1)>0)]
    data_group = data.groupby('Flipon')['Gene ID'].nunique().sort_index()
    data_all = data['Gene ID'].nunique()
    result_df_2 = pd.concat([result_df_2, pd.Series([*data_group.values, data_all], index=[*data_group.index, 'Any flipon'], name=groups)], axis=1)
result_df_2.columns = [['# of genes with ≥1 flipon bound by c. miR'] * result_df_2.shape[1], result_df_2.columns]
result_df_2

result_df_3 = pd.DataFrame()
for groups, mir_group_combination in mir_group_combinations:
    data = flipon_to_data[(flipon_to_data['Gene Type']=='protein_coding') & (flipon_to_data[mir_group_combination].notna().sum(axis=1)>0)]
    data_group = data.groupby('Flipon')['Gene ID'].nunique().sort_index()
    data_all = data['Gene ID'].nunique()
    result_df_3 = pd.concat([result_df_3, pd.Series([*data_group.values, data_all], index=[*data_group.index, 'Any flipon'], name=groups)], axis=1)
result_df_3.columns = [['# of genes with ≥1 flipon bound by c. miR (coding)'] * result_df_3.shape[1], result_df_3.columns]
result_df_3

result_df_4 = pd.DataFrame()
for groups, mir_group_combination in mir_group_combinations:
    data = flipon_to_data[(flipon_to_data['Gene Feature']=="Promoter (<=1kb)")]
    data_group = data.groupby('Flipon')['Gene ID'].nunique().sort_index()
    data_all = data['Gene ID'].nunique()
    result_df_4 = pd.concat([result_df_4, pd.Series([*data_group.values, data_all], index=[*data_group.index, 'Any flipon'], name=groups)], axis=1)
result_df_4.columns = [['# of genes with ≥1 flipon in PP'] * result_df_4.shape[1], result_df_4.columns]
result_df_4

result_df_5 = pd.DataFrame()
for groups, mir_group_combination in mir_group_combinations:
    data = flipon_to_data[(flipon_to_data['Gene Feature']=="Promoter (<=1kb)") & (flipon_to_data['Gene Type']=='protein_coding')]
    data_group = data.groupby('Flipon')['Gene ID'].nunique().sort_index()
    data_all = data['Gene ID'].nunique()
    result_df_5 = pd.concat([result_df_5, pd.Series([*data_group.values, data_all], index=[*data_group.index, 'Any flipon'], name=groups)], axis=1)
result_df_5.columns = [['# of genes with ≥1 flipon in PP (coding)'] * result_df_5.shape[1], result_df_5.columns]
result_df_5

result_df_6 = pd.DataFrame()
for groups, mir_group_combination in mir_group_combinations:
    data = flipon_to_data[(flipon_to_data['Gene Feature']=="Promoter (<=1kb)") & (flipon_to_data[mir_group_combination].notna().sum(axis=1)>0)]
    data_group = data.groupby('Flipon')['Gene ID'].nunique().sort_index()
    data_all = data['Gene ID'].nunique()
    result_df_6 = pd.concat([result_df_6, pd.Series([*data_group.values, data_all], index=[*data_group.index, 'Any flipon'], name=groups)], axis=1)
result_df_6.columns = [['# of genes with ≥1 flipon bound by c. miR in PP'] * result_df_6.shape[1], result_df_6.columns]
result_df_6

result_df_7 = pd.DataFrame()
for groups, mir_group_combination in mir_group_combinations:
    data = flipon_to_data[(flipon_to_data['Gene Feature']=="Promoter (<=1kb)") & (flipon_to_data['Gene Type']=='protein_coding') & (flipon_to_data[mir_group_combination].notna().sum(axis=1)>0)]
    data_group = data.groupby('Flipon')['Gene ID'].nunique().sort_index()
    data_all = data['Gene ID'].nunique()
    result_df_7 = pd.concat([result_df_7, pd.Series([*data_group.values, data_all], index=[*data_group.index, 'Any flipon'], name=groups)], axis=1)
result_df_7.columns = [['# of genes with ≥1 flipon bound by c. miR in PP (coding)'] * result_df_7.shape[1], result_df_7.columns]
result_df_7

display(result_df, result_df_2, result_df_3, result_df_4, result_df_5, result_df_6, result_df_7)

,# of genes with ≥1 flipon,# of genes with ≥1 flipon (coding)
g4,10742,4901
h-dna,8807,3192
sidd,10194,3202
z-dna,10297,4963
Any flipon,21197,8400


# of genes with ≥1 flipon bound by c. miR                      \
                                                   M     MJ     J   M+MJ   
g4                                              5714   5235  1278   7626   
h-dna                                           2094   2744   792   3811   
sidd                                            9902   3500   873   9986   
z-dna                                           4172   5319  1465   6670   
Any flipon                                     15411  12076  3993  17640   

                                 
              M+J   MJ+J M+MJ+J  
g4           6199   5829   7923  
h-dna        2606   3176   4138  
sidd         9921   3980   9998  
z-dna        4804   5831   6990  
Any flipon  15980  13137  17924

# of genes with ≥1 flipon bound by c. miR (coding)              \
                                                            M    MJ     J   
g4                                                       2849  2483   582   
h-dna                                                     735  1022   280   
sidd                                                     3088  1038   252   
z-dna                                                    1950  2626   790   
Any flipon                                               5894  5031  1695   

                                     
            M+MJ   M+J  MJ+J M+MJ+J  
g4          3641  3034  2707   3741  
h-dna       1391   924  1177   1509  
sidd        3120  3098  1177   3128  
z-dna       3251  2309  2897   3425  
Any flipon  6922  6153  5420   7040

# of genes with ≥1 flipon in PP                                \
                                         M    MJ     J  M+MJ   M+J  MJ+J   
g4                                    3498  3498  3498  3498  3498  3498   
h-dna                                 1298  1298  1298  1298  1298  1298   
sidd                                   827   827   827   827   827   827   
z-dna                                 3871  3871  3871  3871  3871  3871   
Any flipon                            5714  5714  5714  5714  5714  5714   

                   
           M+MJ+J  
g4           3498  
h-dna        1298  
sidd          827  
z-dna        3871  
Any flipon   5714

# of genes with ≥1 flipon in PP (coding)                          \
                                                  M    MJ     J  M+MJ   M+J   
g4                                             2769  2769  2769  2769  2769   
h-dna                                           860   860   860   860   860   
sidd                                            377   377   377   377   377   
z-dna                                          3022  3022  3022  3022  3022   
Any flipon                                     3954  3954  3954  3954  3954   

                         
            MJ+J M+MJ+J  
g4          2769   2769  
h-dna        860    860  
sidd         377    377  
z-dna       3022   3022  
Any flipon  3954   3954

# of genes with ≥1 flipon bound by c. miR in PP                   \
                                                         M    MJ    J  M+MJ   
g4                                                    1950  1562  323  2505   
h-dna                                                  241   365   83   508   
sidd                                                   764   212   49   778   
z-dna                                                 1156  1730  471  2228   
Any flipon                                            3224  3010  877  4233   

                               
             M+J  MJ+J M+MJ+J  
g4          2075  1714   2577  
h-dna        312   422    557  
sidd         768   251    782  
z-dna       1449  1939   2384  
Any flipon  3487  3310   4367

# of genes with ≥1 flipon bound by c. miR in PP (coding)        \
                                                                  M    MJ   
g4                                                       1629        1288   
h-dna                                                     175         250   
sidd                                                      341          96   
z-dna                                                     919        1395   
Any flipon                                               2312        2282   

                                          
              J  M+MJ   M+J  MJ+J M+MJ+J  
g4          265  2055  1724  1402   2103  
h-dna        59   358   226   290    391  
sidd         24   349   344   115    352  
z-dna       394  1781  1164  1565   1909  
Any flipon  696  3028  2507  2491   3119

In [135]:
result_df = pd.DataFrame()

data = flipon_to_data.copy()
group = data.groupby('Flipon')['Transcript ID'].nunique().sort_index()
all = data['Transcript ID'].nunique()
result_df = pd.concat([result_df, pd.Series([*group.values, all], index=[*group.index, 'Any flipon'], name='# of transcripts with ≥1 flipon')], axis=1)

data = flipon_to_data[(flipon_to_data['Gene Type']=='protein_coding')]
group = data.groupby('Flipon')['Transcript ID'].nunique().sort_index()
all = data['Transcript ID'].nunique()
result_df = pd.concat([result_df, pd.Series([*group.values, all], index=[*group.index, 'Any flipon'], name='# of transcripts with ≥1 flipon (coding)')], axis=1)

result_df_2 = pd.DataFrame()
for groups, mir_group_combination in mir_group_combinations:
    data = flipon_to_data[(flipon_to_data[mir_group_combination].notna().sum(axis=1)>0)]
    group = data.groupby('Flipon')['Transcript ID'].nunique().sort_index()
    all = data['Transcript ID'].nunique()
    result_df_2 = pd.concat([result_df_2, pd.Series([*group.values, all], index=[*group.index, 'Any flipon'], name=groups)], axis=1)
result_df_2.columns = [['# of transcripts with ≥1 flipon bound by c. miR'] * result_df_2.shape[1], result_df_2.columns]
result_df_2

result_df_3 = pd.DataFrame()
for groups, mir_group_combination in mir_group_combinations:
    data = flipon_to_data[(flipon_to_data['Gene Type']=='protein_coding') & (flipon_to_data[mir_group_combination].notna().sum(axis=1)>0)]
    group = data.groupby('Flipon')['Transcript ID'].nunique().sort_index()
    all = data['Transcript ID'].nunique()
    result_df_3 = pd.concat([result_df_3, pd.Series([*group.values, all], index=[*group.index, 'Any flipon'], name=groups)], axis=1)
result_df_3.columns = [['# of transcripts with ≥1 flipon bound by c. miR (coding)'] * result_df_3.shape[1], result_df_3.columns]
result_df_3

display(result_df, result_df_2, result_df_3)

,# of transcripts with ≥1 flipon,# of transcripts with ≥1 flipon (coding)
g4,11796,5892
h-dna,9097,3456
sidd,10494,3456
z-dna,11377,5985
Any flipon,23869,10844


# of transcripts with ≥1 flipon bound by c. miR               \
                                                         M     MJ     J   
g4                                                    6077   5470  1294   
h-dna                                                 2112   2773   795   
sidd                                                 10183   3530   875   
z-dna                                                 4328   5571  1484   
Any flipon                                           16620  12979  4085   

                                        
             M+MJ    M+J   MJ+J M+MJ+J  
g4           8189   6601   6108   8510  
h-dna        3873   2636   3217   4213  
sidd        10271  10202   4017  10283  
z-dna        7094   5018   6132   7455  
Any flipon  19382  17301  14169  19731

# of transcripts with ≥1 flipon bound by c. miR (coding)        \
                                                                  M    MJ   
g4                                                       3193        2709   
h-dna                                                     751        1048   
sidd                                                     3327        1062   
z-dna                                                    2098        2869   
Any flipon                                               7000        5881   

                                           
               J  M+MJ   M+J  MJ+J M+MJ+J  
g4           598  4175  3417  2971   4297  
h-dna        283  1449   951  1215   1579  
sidd         254  3363  3337  1207   3371  
z-dna        809  3659  2511  3183   3867  
Any flipon  1781  8527  7360  6377   8701

In [145]:
result_df_2 = pd.DataFrame()
for groups, mir_group_combination in mir_group_combinations:
    data = flipon_to_data[(flipon_to_data['Gene Feature'] == "5' UTR") & (flipon_to_data[mir_group_combination].notna().sum(axis=1)>0)]
    group = data.groupby('Flipon')['Transcript ID'].nunique().sort_index()
    all = data['Transcript ID'].nunique()
    result_df_2 = pd.concat([result_df_2, pd.Series([*group.values, all], index=[*group.index, 'Any flipon'], name=groups)], axis=1)
result_df_2.columns = [['# of transcripts with ≥1 flipon bound by c. miR (5\' UTR only)'] * result_df_2.shape[1], result_df_2.columns]
result_df_2.fillna('')

# of transcripts with ≥1 flipon bound by c. miR (5' UTR only)       \
                                                                       M   MJ   
g4                                                          7             4.0   
sidd                                                        1                   
z-dna                                                       2             2.0   
Any flipon                                                  9             6.0   

                                      
              J M+MJ M+J MJ+J M+MJ+J  
g4          4.0    9   7  7.0      9  
sidd               1   1           1  
z-dna              4   2  2.0      4  
Any flipon  4.0   13   9  9.0     13

In [138]:
result_df = pd.DataFrame()

data = flipon_to_data[(flipon_to_data['Gene Feature']=="Promoter (<=1kb)")].copy()
group = data.groupby('Flipon')['Transcript ID'].nunique().sort_index()
all = data['Transcript ID'].nunique()
result_df = pd.concat([result_df, pd.Series([*group.values, all], index=[*group.index, 'Any flipon'], name='# of PP with ≥1 flipon')], axis=1)

data = flipon_to_data[(flipon_to_data['Gene Feature']=="Promoter (<=1kb)") & (flipon_to_data['Gene Type']=='protein_coding')]
group = data.groupby('Flipon')['Transcript ID'].nunique().sort_index()
all = data['Transcript ID'].nunique()
result_df = pd.concat([result_df, pd.Series([*group.values, all], index=[*group.index, 'Any flipon'], name='# of PP with ≥1 flipon (coding)')], axis=1)

result_df_2 = pd.DataFrame()
for groups, mir_group_combination in mir_group_combinations:
    data = flipon_to_data[(flipon_to_data['Gene Feature']=="Promoter (<=1kb)") & (flipon_to_data[mir_group_combination].notna().sum(axis=1)>0)]
    group = data.groupby('Flipon')['Transcript ID'].nunique().sort_index()
    all = data['Transcript ID'].nunique()
    result_df_2 = pd.concat([result_df_2, pd.Series([*group.values, all], index=[*group.index, 'Any flipon'], name=groups)], axis=1)
result_df_2.columns = [['# of PP with ≥1 flipon bound by c. miR'] * result_df_2.shape[1], result_df_2.columns]
result_df_2

result_df_3 = pd.DataFrame()
for groups, mir_group_combination in mir_group_combinations:
    data = flipon_to_data[(flipon_to_data['Gene Feature']=="Promoter (<=1kb)") & (flipon_to_data['Gene Type']=='protein_coding') & (flipon_to_data[mir_group_combination].notna().sum(axis=1)>0)]
    group = data.groupby('Flipon')['Transcript ID'].nunique().sort_index()
    all = data['Transcript ID'].nunique()
    result_df_3 = pd.concat([result_df_3, pd.Series([*group.values, all], index=[*group.index, 'Any flipon'], name=groups)], axis=1)
result_df_3.columns = [['# of PP with ≥1 flipon bound by c. miR (coding)'] * result_df_3.shape[1], result_df_3.columns]
result_df_3

display(result_df, result_df_2, result_df_3)

,# of PP with ≥1 flipon,# of PP with ≥1 flipon (coding)
g4,4080,3330
h-dna,1361,921
sidd,832,382
z-dna,4547,3676
Any flipon,7047,5238


# of PP with ≥1 flipon bound by c. miR                         \
                                                M    MJ    J  M+MJ   M+J   
g4                                           2135  1660  329  2785  2279   
h-dna                                         243   370   83   520   315   
sidd                                          768   213   49   782   772   
z-dna                                        1219  1845  480  2443  1539   
Any flipon                                   3624  3397  900  4954  3951   

                         
            MJ+J M+MJ+J  
g4          1831   2873  
h-dna        430    572  
sidd         252    786  
z-dna       2080   2620  
Any flipon  3750   5125

# of PP with ≥1 flipon bound by c. miR (coding)                   \
                                                         M    MJ    J  M+MJ   
g4                                                    1807  1383  271  2326   
h-dna                                                  177   255   59   370   
sidd                                                   345    97   24   353   
z-dna                                                  980  1508  403  1992   
Any flipon                                            2699  2661  719  3728   

                               
             M+J  MJ+J M+MJ+J  
g4          1921  1515   2389  
h-dna        229   298    406  
sidd         348   116    356  
z-dna       1250  1702   2139  
Any flipon  2957  2919   3855

In [137]:
print('# of genes with >= 1 flipon bound by miR')

result_df = pd.DataFrame()
for groups, mir_group_combination in mir_group_combinations:
    data = flipon_to_data[(flipon_to_data[mir_group_combination].notna().sum(axis=1)>0)]
    data_group = data.groupby('Flipon')['Gene ID'].nunique().sort_index()
    data_all = data['Gene ID'].nunique()
    result_df = pd.concat([result_df, pd.Series([*data_group.values, data_all], index=[*data_group.index, 'Any flipon'], name=groups)], axis=1)
result_df

# of genes with >= 1 flipon bound by miR


,M,MJ,J,M+MJ,M+J,MJ+J,M+MJ+J
g4,5714,5235,1278,7626,6199,5829,7923
h-dna,2094,2744,792,3811,2606,3176,4138
sidd,9902,3500,873,9986,9921,3980,9998
z-dna,4172,5319,1465,6670,4804,5831,6990
Any flipon,15411,12076,3993,17640,15980,13137,17924
